<a href="https://colab.research.google.com/github/Roythescientist/Dog-breeds-prediction-Using-Supervised-Learning-/blob/main/Dog_Breeds_(Supervised_Learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#importing files from Kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"austineroy","key":"5d38658b02b39c09eb80bb97e77cd56e"}'}

Install kaggle API

In [4]:
#Install Kaggle API clent
!pip install -q kaggle

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

#this permision avoids warning on kaggle tools startup
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#creating a new directory and changing the current worki ng directory
!mkdir dog_dataset
%cd dog_dataset

In [ ]:
#searching for datasets in kaggle
!kaggle datasets list -s dog-breed-id-from-comp

In [ ]:
#Downloading dataset and coming out
!kaggle datasets download catherinehorng/dogbreedidfromcomp --force
%cd ..

In [ ]:
#unzip the zipped file and after uploading it then remove it from thr folder
!unzip dog_dataset/dogbreedidfromcomp.zip -d dog_dataset
!rm dog_dataset/dogbreedidfromcomp.zip
!rm dog_dataset/sample_submission.csv

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
import seaborn as sns

from keras.preprocessing import image
from tqdm import tqdm
from keras.models import Sequential
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
#Read the labels and cheching on size and shape
all_labels=pd.read_csv('dog_dataset/labels.csv')
print(all_labels.shape)
print(all_labels.head())

In [ ]:
#visualize the number of each breed
all_breeds=all_labels['breed']
breed_count=all_breeds.value_counts()
breed_count.head()

In [ ]:
#selecting the  breeds
class_names=['scottish_deerhound','maltese_dog','afghan_hound']
labels=all_labels[(all_labels['breed'].isin(class_names))]
labels=labels.reset_index()
labels.head()

In [ ]:
#creating numpy by matrix
X_data=np.zeros((len(labels),224,224,3),dtype='float32')

#one hot encoding
Y_data=label_binarize(labels['breed'],classes=class_names)

#Reasding and converting numpy array and normalizing dataset
for i in tqdm(range(len(labels))):
  img=image.load_img('dog_dataset/train/%s.jpg'%labels['id'][i],target_size=(224,224))
  img=image.img_to_array(img)
  x=np.expand_dims(img.copy(),axis=0)
  X_data[i]=x/255.0

#printing the train image and one hot encode shape and size
print('\nTrain Images Shape: ',X_data.shape,'size: {:,}'.format(X_data.size))
print('One -hot encoded output shape: ',Y_data.shape,' size: {:,}'.format(Y_data.size))

In [ ]:
#Building a model
model=Sequential()

model.add(Conv2D(filters=64,kernel_size=(5,5),activation='relu',input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu',kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=16,kernel_size=(7,7),activation='relu',kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=8,kernel_size=(5,5),activation='relu',kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128,activation='relu',kernel_regularizer='l2'))
model.add(Dense(64,activation='relu',kernel_regularizer='l2'))
model.add(Dense(len(class_names),activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer=Adam(0.0001),metrics=['accuracy'])
model.summary()

In [16]:
#splitting the data into training and testing data sets
X_train_and_val,X_test,Y_train_and_val,Y_test =train_test_split(X_data,Y_data,test_size=0.1)

#splitting the training dataset into training and validation datasets
X_train,X_val,Y_train,Y_val = train_test_split(X_train_and_val,Y_train_and_val,test_size=0.2)

In [ ]:
#Training th model
epochs=150
batch_size=100

history=model.fit(X_train,Y_train,batch_size=batch_size,epochs=epochs,
                  validation_data=(X_val,Y_val))

In [ ]:
#plot the traning history
plt.figure(figsize=(12,5))
plt.plot(history.history['accuracy'],color='r')
plt.plot(history.history['val_accuracy'],color='b')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train','val'])

plt.show()

In [ ]:
Y_predict=model.predict(X_test)
score=model.evaluate(X_test,Y_test)

print('Accuracy over the test set: \n',round((score[1]*100),2),'%')

In [ ]:
#plotting image to compare
plt.imshow(X_test[1,:,:,:])
plt.show()

#Finding the maximum value of prediction list and comparing original value vs predicted
print('Original: ',labels['breed'][np.argmax(Y_test[1])])
print('Predicted:',labels['breed'][np.argmax(Y_predict[1])])